# 소실점

In [3]:
import cv2
import numpy as np

def grayscale(img): # 흑백이미지로 변환
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold): # Canny알고리즘
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kerner_size): # 가우시안 필터
    return cv2.GaussianBlur(img, (kerner_size, kerner_size), 0)

def region_of_interest(img, vertices, color3 = (255, 255, 255), color1 = 255):
    #ROI setting
    mask = np.zeros_like(img) # img와 같은 크기의 빈 이미지

    if len(img.shape) > 2 : #if color image (3 channels)
        color = color3
    else : #if black&white image ( 1 channel)
        color = color1
    
    #vertices에 정한 점들로 이루어진 다각형 부분(ROI부분)을 color로 채움
    cv2.fillPoly(mask, vertices, color)
    
    #이미지와 color로 체워진 ROI를 합침
    ROI_img = cv2.bitwise_and(img, mask)
    return ROI_img

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap): #허프 변환
    lines = cv2.HoughLines(img, rho, theta, threshold, np.array([]))#, minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype = np.uint8)
    draw_lines(line_img, lines)
    
    return line_img

def weighted_img(img, initial_img, a = 1, b = 1., c = 0.): #두 이지미 합치기
    return cv2.addWeighted(initial_img, a, img, b, c)


In [2]:
def get_fitline(img, f_lines):
    lines = np.squeeze(f_lines)
    lines = lines.reshape(lines.shape[0] * 2, 2)
    rows, cols = img.shape[:2]
    output = cv2.fitLine(lines, cv2.DIST_L2, 0, 0.01, 0.01)
    
    vx, vy, x, y = output[0], output[1], output[2], output[3]
    x1, y1 = int(((img.shape[0] - 1) - y) / vy * vx + x), img.shape[0] - 1
    x2, y2 = int(((img.shape[0]/2 + 100) - y) / vy * vx + x), int(img.shape[0] / 2 + 100)
    
    result = [x1,y1, x2, y2]
    return result
def draw_lines(img, lines, color=[0,0,255], thickness = 2): #선그리기
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
#def draw_lines(img, lines, color=[0,0,255], thickness = 10): #선그리기
#   cv2.line(img, (lines[0], lines[1]), (lines[2], lines[3]), color, thickness)

In [4]:

image = cv2.imread("slope_test.jpg")
    
height, width = image.shape[:2]

gray_img = grayscale(image) # 흑백이미지로 변환

blur_img = gaussian_blur(gray_img, 3) # Blur 효과

canny_img = canny(blur_img, 70, 210) # Canny Edge 알고리ㅑ

#cv2.imshow('canny', canny_img) # Canny 이미지 출력

vertices = np.array([[(50, height), (width/2 - 45, height/2 + 60), (width/2 + 45, height/2 + 60), (width-50, height)]], dtype=np.int32)

roi_img = region_of_interest(canny_img, vertices) # vertices에 정한 점들을 기준으로 ROI이미지 생성
#cv2.imshow('roi', roi_img)

line_arr = hough_lines(roi_img, 1, 1 * np.pi/180, 30, 10 ,20) #허프 변환
#line_arr = np.squeeze(line_arr)

cv2.imshow('res', weighted_img(line_arr, image))

cv2.waitKey(0)


ValueError: not enough values to unpack (expected 4, got 2)

In [5]:
print(image.shape)
print(line_arr.shape)

(540, 960, 3)


NameError: name 'line_arr' is not defined

# 동영상 적용

In [5]:

cap = cv2.VideoCapture("solidWhiteRight.mp4")

while (cap.isOpened()):
    ret, image = cap.read()
    if not ret:
        break
    height, width = image.shape[:2]

    gray_img = grayscale(image) # 흑백이미지로 변환

    blur_img = gaussian_blur(gray_img, 3) # Blur 효과

    canny_img = canny(blur_img, 70, 210) # Canny Edge 알고리ㅑ

    cv2.imshow('canny', canny_img) # Canny 이미지 출력

    vertices = np.array([[(50, height), (width/2 - 45, height/2 + 60), (width/2 + 45, height/2 + 60), (width-50, height)]], dtype=np.int32)

    roi_img = region_of_interest(canny_img, vertices) # vertices에 정한 점들을 기준으로 ROI이미지 생성
    #cv2.imshow('roi', roi_img)

    line_arr = hough_lines(roi_img, 1, 1 * np.pi/180, 30, 10 ,20) #허프 변환
    line_arr = np.squeeze(line_arr)

    #기울기 구하기
    slope_degree = (np.arctan2(line_arr[:,1] - line_arr[:,3], line_arr[:,0] - line_arr[:, 2]) * 180) / np.pi

    #수평 기울기 제한
    line_arr = line_arr[np.abs(slope_degree) < 160]
    slope_degree = slope_degree[np.abs(slope_degree) < 160]

    #수직 기울기 제한
    line_arr = line_arr[np.abs(slope_degree) > 95]
    slope_degree = slope_degree[np.abs(slope_degree) > 95]

    #필터링 직선 버리기
    L_lines, R_lines = line_arr[(slope_degree > 0), :], line_arr[(slope_degree < 0), :]
    temp = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    L_lines, R_lines = L_lines[:,None], R_lines[:,None]

    #왼쪽 오른쪽 각각 대표선 구하기
    left_fit_line = get_fitline(image, L_lines)
    right_fit_line = get_fitline(image, R_lines)

    #대표선 그리기
    draw_lines(temp, left_fit_line)
    draw_lines(temp, right_fit_line)

    result = weighted_img(temp, image)
    cv2.imshow('result', result)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release
cap.release()
cv2.destroyAllWindows()
